## <font color='green'> <div align="center">In the name of God </div></font>

### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> </font>

# GMM Quadratic Classifier on MNIST with reg = 0

##### Importing general modules.

In [66]:
import numpy as np
from sklearn import mixture
import scipy.io as sio
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

### Preparing matplotlib to genrate high-quality images for the paper

In [67]:
%matplotlib inline
mpl.rcParams['figure.dpi']= 600

In [68]:
C  = 10
K = 6
L = C * K
width = 28
height = 28
reg = 0

### Load Training and Testing Data and Normalize them

In [6]:
XTrain = sio.loadmat ('../../datasets/mnist/MnistTrainX')['MnistTrainX']
XTrain = XTrain / 255
yTrain = sio.loadmat ('../../datasets/mnist/MnistTrainY')['MnistTrainY']
XTest = sio.loadmat ('../../datasets/mnist/MnistTestX')['MnistTestX']
XTest = XTest / 255
yTest = sio.loadmat ('../../datasets/mnist/MnistTestY')['MnistTestY']
N, dim = XTrain.shape

### Loading GMM data computed on Google Colab

In [7]:
clusters = np.zeros(N, dtype = int)
with open('GMM_MNIST_6.txt', 'r') as f:
    for i in range (N):
        s = f.readline()
        clusters[i] = int(s)

### Compute means and covariances

In [9]:
means = np.zeros([L, dim])
cov = np.zeros([L, dim, dim])
for i in range (L):
    idx = clusters == i
    means[i,:] = np.mean(XTrain[idx,:], axis = 0)
    cov[i,:,:] = np.cov (XTrain[idx,:].T)    

### Compute priors

In [49]:
Nck = np.bincount(clusters)
Pck = Nck / N

In [62]:
logProb = np.zeros ([N,L])
for i in tqdm(range (L)):
    eigvals, eigvecs = np.linalg.eig(cov[i,:,:])
    idx = eigvals > 0.0001
    eigvals = eigvals[idx] + reg
    eigvecs = eigvecs[:,idx]
    sigmaHalfInv = eigvecs @ np.diag(np.sqrt(1/eigvals)) @ eigvecs.T
    X = (XTrain - means[i,:]) @ sigmaHalfInv
    X = X ** 2
    d2 = np.sum(X, axis = 1)
    logProb[:,i] = np.log(Pck[i]) - 1 / 2 * d2
prediction = np.argmax (logProb, axis = 1) // K
score = np.sum(prediction == yTrain.squeeze())/ N
print ('Training Accuracy = ', score)

  0%|                                                                                           | 0/60 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  # This is added back by InteractiveShellApp.init_path()
100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [11:30<00:00, 12.85s/it]


Training Accuracy =  0.9662


In [63]:
NTest = XTest.shape[0]
logProb = np.zeros ([NTest,L])
for i in tqdm(range (L)):
    eigvals, eigvecs = np.linalg.eig(cov[i,:,:])
    idx = eigvals > 0.0001
    eigvals = eigvals[idx] + reg
    eigvecs = eigvecs[:,idx]
    sigmaHalfInv = eigvecs @ np.diag(np.sqrt(1/eigvals)) @ eigvecs.T
    X = (XTest - means[i,:]) @ sigmaHalfInv
    X = X ** 2
    d2 = np.sum(X, axis = 1)
    logProb[:,i] = np.log(Pck[i]) - 1 / 2 * d2
prediction = np.argmax (logProb, axis = 1) // K
score = np.sum(prediction == yTest.squeeze())/ NTest
print ('Testing Accuracy = ', score)

  0%|                                                                                           | 0/60 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':
100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [02:41<00:00,  2.35s/it]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


Testing Accuracy =  0.0


In [64]:
score = np.sum(prediction == yTest.squeeze())/ NTest
print ('Testing Accuracy = ', score)

Testing Accuracy =  0.8805


### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> </font>